<a href="https://colab.research.google.com/github/ahmedmgelwan/scraping_google_images/blob/main/scraping%20google%20images%20with%20selenium.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# %%shell
# # Ubuntu no longer distributes chromium-browser outside of snap
# #
# # Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# # Add debian buster
# cat > /etc/apt/sources.list.d/debian.list <<'EOF'
# deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
# deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
# deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
# EOF

# # Add keys
# apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
# apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
# apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

# apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
# apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
# apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# # Prefer debian repo for chromium* packages only
# # Note the double-blank lines between entries
# cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
# Package: *
# Pin: release a=eoan
# Pin-Priority: 500


# Package: *
# Pin: origin "deb.debian.org"
# Pin-Priority: 300
# 'https://www.google.com/search?sca_esv=564207690&q=mo+salah&tbm=isch&source=lnms&sa=X&ved=2ahUKEwjO4Z3n_qCBAxVlVKQEHcFwCKcQ0pQJegQICxAB&biw=1225&bih=734&dpr=1'

# Package: chromium*
# Pin: origin "deb.debian.org"
# Pin-Priority: 700
# EOF

# # Install chromium and chromium-driver
# apt-get update
# apt-get install chromium chromium-driver

# Install selenium
# pip install selenium

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import time
import urllib
import os

In [3]:
def save_img(url,name):
    img = urllib.request.urlopen(url)
    out = open(name,'wb')
    out.write(img.read())
    out.close()

In [ ]:
service = Service(executable_path="/usr/bin/chromedriver")
options = webdriver.ChromeOptions()

options.add_argument('--headless')
options.add_argument('--no-sandbox')
driver = webdriver.Chrome(service=service, options=options)
url = input('URL - ')
driver.get(url)
time.sleep(5)
full_xpath = '/html/body/div[2]/c-wiz/div[3]/div[1]/div/div/div/div/div[1]/div[1]/span/div[1]/div[1]/div[1]'
# pics = driver.find_element('xpath',full_xpath)
pics = driver.find_elements(By.CLASS_NAME, 'rg_i')
name = driver.title.split(' - ')[0].title()
links = []
x = 0
last_height=0
print(name)
try:
    os.chdir('/content/drive/MyDrive/Projects/web_scraping/google_images')
    os.mkdir(name)
    os.chdir(name)
except:
    exit()
while True:

    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(40)

    for i,pic in enumerate(pics):
        try:
            img_link = pic.get_attribute('src')
            if img_link not in links:
                links.append(img_link)
                file_name=str(x+1).zfill(3)+'.jpg'
                save_img(img_link,file_name)

                x+=1
                print(f'{file_name} has saved.')

        except:
            print(f'{file_name} Failed.')
    new_height = driver.execute_script("return document.body.scrollHeight")
    print(new_height)
    if new_height == last_height:
        exit()
    last_height = new_height
print('All is done')
driver.quit()